In [2]:
import logging
import numpy as np

import matplotlib.pyplot as plt
from importlib import reload
import random

import tensorflow as tf
import importlib
import MCAgent as agent
import Model as Con4
import All_In_One_environment as env
from tensorflow.python.client import device_lib
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, BatchNormalization, Activation, LeakyReLU, add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

2021-12-05 18:32:01.825379: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [21]:
#local imports reload
importlib.reload(agent)
importlib.reload(Con4)
importlib.reload(env)

<module 'All_In_One_environment' from '/home/jevans40/AI/AIClass2021/Project/Group2-s_UW_Intro_AI_Project/Project_attempt_one/Project_attempt_one/All_In_One_environment.py'>

In [23]:
#settings:
#Settings
inputDim = (6,7,2)
outputDim = (7)
filterSize = 75
kernelDim = (3,3)
convLayers = 3
regConst = 0.0001
learningRate = 0.001
iteration = 1

Memory = []

In [22]:
def RunTwoAgents(Agent1,Agent2, Games, Verbosity = 0):
    memories = []
    for game in range(Games):
        print("Playing game {} out of {}".format(game,Games))
        winner = PlayGame(Agent1,Agent2,Verbosity)
        memories += Agent1.GetMemory()
        memories += Agent2.GetMemory()
        Agent1.ClearMemory()
        Agent2.ClearMemory()   

        tmp = Agent2
        Agent2 = Agent1
        Agent1 = tmp
    return memories
    #Return memory to train on

#Verbosity controls how much information to output
#0: None
#1: Winner/Loser
#2: Memory Output
def PlayGame(Agent1,Agent2,Verbosity=0):
    Agent1.SetPlayer(-1)
    Agent2.SetPlayer(1)
    done = False
    memories = []
    newGame = env.ConnectFour(1)
    num = random.randint(0,6)
    newGame.step(num,1)
    while not done:
        action = Agent1.GetAction(newGame)
        #Add some level of randomness so that we can get more varied cases
        nu = np.random.dirichlet([1]*7)/20
        action = action + nu
        newState,reward,done,info = newGame.step_array(action,1)
        memories.append(action)
        if Verbosity > 1:
            print("Player -1 took the action\n{}".format(action))
        if done:
            if Verbosity > 0:
                print("Player -1 has won the game!")
            return -1

        action = Agent2.GetAction(newGame)
        nu = np.random.dirichlet([1]*7)/20
        action = action + nu
        newState,reward,done,info = newGame.step_array(action,-1)
        if Verbosity > 1:
            print("Player 1 took the action\n{}".format(action))
        if done:
            if Verbosity > 0:
                print("Player 1 has won the game!")
            return 1
        if Verbosity > 2:
            print(np.array(newGame.get_current_state()).reshape(6,7))
    
    #Return game result

def TrainModel(Memories, toFitAgent):
    for i in range(5):
        minbatch = random.sample(Memories,min(250,len(Memories)))

        trainingStates = np.array([Con4.ReshapeToModel(item[0],item[3]) for item in Memories])
        trainingTargets = np.array([item[1] for item in Memories])

        fit = toFitAgent.model.fit(trainingStates,trainingTargets,epochs=5,verbose=1,validation_split=0,batch_size = 32)

        toFitAgent.model.save("ModelCheckPointStart")
    return round(fit.history["loss"][0],4)

In [24]:
testModel = Con4.CN4Model(inputDim,outputDim,filterSize,kernelDim,regConst,convLayers,learningRate)
testModel.InitModel()


In [25]:
Agent1 = agent.ReinforcementAgent(testModel,-1)
Agent2 = agent.ReinforcementAgent(testModel,1)

In [ ]:
Mem = RunTwoAgents(Agent1, Agent2,5, Verbosity=0)
print(Mem)

In [27]:
TrainModel(Mem,testModel)

Epoch 1/5
4/4 [==============================] - 7s 2s/step - loss: 2.1804
Epoch 2/5
4/4 [==============================] - 0s 6ms/step - loss: 1.8092
Epoch 3/5
4/4 [==============================] - 0s 5ms/step - loss: 1.7521
Epoch 4/5
4/4 [==============================] - 0s 7ms/step - loss: 1.8027
Epoch 5/5
4/4 [==============================] - 0s 5ms/step - loss: 1.7665
INFO:tensorflow:Assets written to: ModelCheckPointStart/assets
Epoch 1/5
4/4 [==============================] - 0s 5ms/step - loss: 1.7806
Epoch 2/5
4/4 [==============================] - 0s 5ms/step - loss: 1.7568
Epoch 3/5
4/4 [==============================] - 0s 5ms/step - loss: 1.7528
Epoch 4/5
4/4 [==============================] - 0s 5ms/step - loss: 1.7435
Epoch 5/5
4/4 [==============================] - 0s 8ms/step - loss: 1.7404
INFO:tensorflow:Assets written to: ModelCheckPointStart/assets
Epoch 1/5
4/4 [==============================] - 0s 5ms/step - loss: 1.7220
Epoch 2/5
4/4 [========================

1.6385

In [28]:
while True:
    Mem = RunTwoAgents(Agent1, Agent2,20, Verbosity=0)
    TrainModel(Mem,testModel)

Playing game 0 out of 20
delta
[-0.01109292 -0.00508166 -0.00956686 -0.03041998  0.06514186  0.06041918
 -0.06960631]
delta
[-2.48898827e-02  5.21130249e-05  7.23250748e-03  1.05333680e-02
  3.54021543e-02  1.71862289e-04 -2.86464090e-02]
delta
[-0.00501045  0.00872139  0.0200432   0.00323084  0.05308543  0.01119151
 -0.09142472]
delta
[-0.02386167  0.00112075  0.00570278  0.00911944  0.03419349  0.00359557
 -0.03001324]
delta
[-0.00297408  0.02196791  0.01888416  0.02597966  0.06715268  0.02446296
 -0.15565033]
Playing game 1 out of 20
delta
[ 0.03383521  0.05877721  0.06336845 -0.03449014 -0.04504151 -0.03988719
 -0.03696289]
delta
[-0.05064392 -0.00959604 -0.02553923  0.01395454  0.04974597  0.01920561
  0.00269746]
delta
[ 0.05555318 -0.03762842  0.08508641 -0.02973632  0.05345918  0.02301646
 -0.15040533]
delta
[-0.00859813 -0.00858827 -0.01186751  0.01899822  0.01648988  0.01359163
 -0.02018406]
delta
[-0.15148477 -0.14156596  0.29982712  0.2252662   0.06270797 -0.12404773
 -0.17